# naver crawling

In [1]:
# 기본
import requests
from bs4 import BeautifulSoup
import re
import time
from io import BytesIO
import pandas as pd
import datetime
import json
import glob
import os

In [2]:
#  기초정보
# 0은 월요일, 1은 화요일, 2는 수요일, 3은 목요일, 4는 금요일 ,5는 토요일 ,6은 일요일
weekday_str = datetime.datetime.now().weekday() # 주 :일자로 변환
if weekday_str == 6 :
    dt_time = datetime.datetime.now() - datetime.timedelta(days=2)
elif weekday_str == 5 :
    dt_time = datetime.datetime.now() - datetime.timedelta(days=1)
else:
    dt_time = datetime.datetime.now()

dt_str = datetime.datetime.strftime(dt_time, '%Y%m%d')
print('작업일자 ==> ', dt_str)

작업일자 ==>  20220415


In [3]:
# 폴더

folder_nm = '주식_' + dt_str
path = './' + folder_nm
if os.path.isdir(path) == False : os.mkdir(folder_nm) 

## (기준정보)업종별 시세

In [4]:
global upjong_lists
upjong_lists = []
file_nm = '업종상세_'+dt_str+'.xlsx'
file_json_temp = '업종상세.json'

def get_upjong_detail(idx, url, upjongNm):
    # print(idx, url, upjongNm)
    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')
    upjong_details = soup.find_all('div', attrs={'class':'name_area'})
    for upjong_detail in upjong_details:
        code_nm = upjong_detail.find('a').get_text()
        code = upjong_detail.find('a')['href'].split('=')[1]
        # print(code, code_nm, upjongNm)
        upjong_lists.append({'종목코드':code, '종목명':code_nm, '업종':upjongNm})

    
url = 'https://finance.naver.com/sise/sise_group.naver?type=upjong'
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
res = requests.get(url)
res.raise_for_status()
url_basic = 'https://finance.naver.com'
soup = BeautifulSoup(res.text, 'lxml')
upjongs = soup.find('table', attrs={'class':'type_1'}).find_all('a')
for idx, upjong in enumerate(upjongs):
    url = url_basic+ upjong['href']
    upjongNm = upjong.get_text().strip()
    
    get_upjong_detail(idx, url, upjongNm)
    # time.sleep(1.0)
 
with open(folder_nm+'/'+ file_json_temp, 'w') as outfile:
    json.dump(upjong_lists, outfile )
    
print('upjong complate ==> {}'.format(file_nm))

upjong complate ==> 업종상세_20220415.xlsx


In [5]:
with open(folder_nm+'/'+ file_json_temp,'r') as json_file:
    json_data = json.load(json_file)
    
# print(json_data)
df_upjong = pd.json_normalize(json_data)
df_upjong.to_excel(folder_nm+'/'+ file_nm)
print('upjong complate ==> {}'.format(file_nm))

upjong complate ==> 업종상세_20220415.xlsx


## (기준정보)테마

In [6]:
global theme_lists
theme_lists = []
file_theme_nm = '테마_'+dt_str+'.xlsx'
file_json_theme_temp = '테마.json'


def get_theme_detail(idx, url, themeNm):
    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')
    theme_details = soup.find_all('div', attrs={'class':'name_area'})

    for theme_detail in theme_details:
        code = theme_detail('a')[0]['href'].split('=')[1]
        code_nm = theme_detail.get_text().replace('*','')
        theme_lists.append({'종목코드':code, '종목명':code_nm, '테마':themeNm})
    
    
# 1~7 page
for i in range(1,8):
    url = 'https://finance.naver.com/sise/theme.naver?&page={}'.format(i)
    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
    res = requests.get(url)
    res.raise_for_status()
    url_basic = 'https://finance.naver.com'
    soup = BeautifulSoup(res.text, 'lxml')
    themes = soup.find_all('td', attrs={'class':'col_type1'})

    for idx, theme in enumerate(themes):
        url = url_basic + theme('a')[0]['href']
        themeNm = theme.get_text()
        get_theme_detail(idx, url, themeNm)
        # time.sleep(1.0)

with open(folder_nm+'/'+ file_json_theme_temp, 'w') as outfile:
    json.dump(theme_lists, outfile)
        
# print(theme_lists)

print('theme complate ==> {}'.format(file_json_theme_temp))

theme complate ==> 테마.json


In [7]:
with open(folder_nm+'/'+ file_json_theme_temp,'r') as json_file:
    json_data = json.load(json_file)
    
# print(json_data)
df_theme = pd.json_normalize(json_data)
df_theme.to_excel(folder_nm+'/'+ file_theme_nm)
print('theme complate ==> {}'.format(file_theme_nm))

theme complate ==> 테마_20220415.xlsx
